In [113]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col, round, mean
from pyspark.ml.feature import StringIndexer , VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [93]:
spark = SparkSession.builder.appName("MachineLearning").getOrCreate()


In [94]:
data = spark.read.format('csv').options(header = 'True', inferSchema = True).load('heart_disease.csv')
data.toPandas()

,ID,Age,Sex,Angina,Blood_Pressure,Cholesterol,Glycemia,ECG,Heart_Rate,Angina_After_Sport,ECG_Angina,ECG_Slope,Fluoroscopy,Thalassaemia,Disease
0,1,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,2,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,3,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,4,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,5,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,299,45.0,1.0,1.0,110.0,264.0,0.0,0.0,132.0,0.0,1.2,2.0,0.0,7.0,1
299,300,68.0,1.0,4.0,144.0,193.0,1.0,0.0,141.0,0.0,3.4,2.0,2.0,7.0,2
300,301,57.0,1.0,4.0,130.0,131.0,0.0,0.0,115.0,1.0,1.2,2.0,1.0,7.0,3
301,302,57.0,0.0,2.0,130.0,236.0,0.0,2.0,174.0,0.0,0.0,2.0,1.0,3.0,1


In [95]:
for col_name in data.columns:
    print(col_name, ": ", data.filter(data[col_name] == "?" ).count())

ID :  0
Age :  0
Sex :  0
Angina :  0
Blood_Pressure :  0
Cholesterol :  0
Glycemia :  0
ECG :  0
Heart_Rate :  0
Angina_After_Sport :  0
ECG_Angina :  0
ECG_Slope :  0
Fluoroscopy :  4
Thalassaemia :  2
Disease :  0


In [96]:
#supprimer
data = data.filter((data.Fluoroscopy != "?")& (data.Thalassaemia != "?"))

In [97]:
data = data.withColumn("Sex", when(col("Sex") == 0, "Female").otherwise("Male"))
data = data.withColumn("Angina",when(col("Angina") == 1, "Stable angina").when(col("Angina") == 2,"Unstable angina").when(col("Angina")==3, "Other pains").otherwise("Asymptomatic"))
data = data.withColumn("Sex", when(col("Sex") == 0, "Female").otherwise("Male"))
data = data.withColumn("Angina", when(col("Angina") == 1, "Stable angina").when(col("Angina") == 2, "Unstable angina").when(col("Angina") == 3, "Other pains").otherwise("Asymptomatic"))
data = data.withColumn("Glycemia", when(col("Glycemia") == 0, "Less than 120 mg/dl").otherwise("More than 120 mg/dl"))
data = data.withColumn("ECG", when(col("ECG") == 0, "Normal").when(col("ECG") == 1, "Anomalies").otherwise("Hypertrophy"))
data = data.withColumn("Angina_After_Sport", when(col("Angina_After_Sport") == 0, "No").otherwise("Yes"))
data = data.withColumn("ECG_Slope", when(col("ECG_Slope") == 1, "Rising").when(col("ECG_Slope") == 2, "Stable").otherwise("Falling"))
data = data.withColumn("Fluoroscopy", when(col("Fluoroscopy") == "0.0", "No anomaly").when(col("Fluoroscopy") == "1.0", "Low").when(col("Fluoroscopy") == "2.0", "Medium").otherwise("High"))
data = data.withColumn("Thalassaemia", when(col("Thalassaemia") == "3.0", "No").when(col("Thalassaemia") == "6.0", "Thalassaemia under control").otherwise("Unstable Thalassaemia"))
data = data.withColumn("Disease", when(col("Disease") == 0, "No").otherwise("Yes"))


data.toPandas()

,ID,Age,Sex,Angina,Blood_Pressure,Cholesterol,Glycemia,ECG,Heart_Rate,Angina_After_Sport,ECG_Angina,ECG_Slope,Fluoroscopy,Thalassaemia,Disease
0,1,63.0,Male,Asymptomatic,145.0,233.0,More than 120 mg/dl,Hypertrophy,150.0,No,2.3,Falling,No anomaly,Thalassaemia under control,No
1,2,67.0,Male,Asymptomatic,160.0,286.0,Less than 120 mg/dl,Hypertrophy,108.0,Yes,1.5,Stable,High,No,Yes
2,3,67.0,Male,Asymptomatic,120.0,229.0,Less than 120 mg/dl,Hypertrophy,129.0,Yes,2.6,Stable,Medium,Unstable Thalassaemia,Yes
3,4,37.0,Male,Asymptomatic,130.0,250.0,Less than 120 mg/dl,Normal,187.0,No,3.5,Falling,No anomaly,No,No
4,5,41.0,Male,Asymptomatic,130.0,204.0,Less than 120 mg/dl,Hypertrophy,172.0,No,1.4,Rising,No anomaly,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,298,57.0,Male,Asymptomatic,140.0,241.0,Less than 120 mg/dl,Normal,123.0,Yes,0.2,Stable,No anomaly,Unstable Thalassaemia,Yes
293,299,45.0,Male,Asymptomatic,110.0,264.0,Less than 120 mg/dl,Normal,132.0,No,1.2,Stable,No anomaly,Unstable Thalassaemia,Yes
294,300,68.0,Male,Asymptomatic,144.0,193.0,More than 120 mg/dl,Normal,141.0,No,3.4,Stable,Medium,Unstable Thalassaemia,Yes
295,301,57.0,Male,Asymptomatic,130.0,131.0,Less than 120 mg/dl,Normal,115.0,Yes,1.2,Stable,Low,Unstable Thalassaemia,Yes


In [98]:
data.describe().show()

+-------+------------------+-----------------+----+------------+------------------+------------------+-------------------+---------+------------------+------------------+------------------+---------+-----------+--------------------+-------+
|summary|                ID|              Age| Sex|      Angina|    Blood_Pressure|       Cholesterol|           Glycemia|      ECG|        Heart_Rate|Angina_After_Sport|        ECG_Angina|ECG_Slope|Fluoroscopy|        Thalassaemia|Disease|
+-------+------------------+-----------------+----+------------+------------------+------------------+-------------------+---------+------------------+------------------+------------------+---------+-----------+--------------------+-------+
|  count|               297|              297| 297|         297|               297|               297|                297|      297|               297|               297|               297|      297|        297|                 297|    297|
|   mean|150.67340067340066|54.54208

In [99]:
data.groupBy('Disease').count().withColumn('Percentage',round((col('count') / data.count()) * 100, 2)).show()
data.groupBy('Sex').count().withColumn('Percentage',round((col('count') / data.count()) * 100, 2)).show()
data.groupBy('Angina').count().withColumn('Percentage',round((col('count') / data.count()) * 100, 2)).show()
data.groupBy('Glycemia').count().withColumn('Percentage',round((col('count') / data.count()) * 100, 2)).show()

+-------+-----+----------+
|Disease|count|Percentage|
+-------+-----+----------+
|     No|  160|     53.87|
|    Yes|  137|     46.13|
+-------+-----+----------+

+----+-----+----------+
| Sex|count|Percentage|
+----+-----+----------+
|Male|  297|     100.0|
+----+-----+----------+

+------------+-----+----------+
|      Angina|count|Percentage|
+------------+-----+----------+
|Asymptomatic|  297|     100.0|
+------------+-----+----------+

+-------------------+-----+----------+
|           Glycemia|count|Percentage|
+-------------------+-----+----------+
|More than 120 mg/dl|   43|     14.48|
|Less than 120 mg/dl|  254|     85.52|
+-------------------+-----+----------+



In [100]:
data.select(data['Age']).summary().show()

+-------+-----------------+
|summary|              Age|
+-------+-----------------+
|  count|              297|
|   mean|54.54208754208754|
| stddev|9.049735681096765|
|    min|             29.0|
|    25%|             48.0|
|    50%|             56.0|
|    75%|             61.0|
|    max|             77.0|
+-------+-----------------+



In [101]:

data.groupBy('Sex', 'Disease').count().show()
data.groupBy('Glycemia', 'Disease').count().show()
data.groupBy('Angina', 'Disease').count().show()

+----+-------+-----+
| Sex|Disease|count|
+----+-------+-----+
|Male|     No|  160|
|Male|    Yes|  137|
+----+-------+-----+

+-------------------+-------+-----+
|           Glycemia|Disease|count|
+-------------------+-------+-----+
|More than 120 mg/dl|    Yes|   20|
|Less than 120 mg/dl|    Yes|  117|
|Less than 120 mg/dl|     No|  137|
|More than 120 mg/dl|     No|   23|
+-------------------+-------+-----+

+------------+-------+-----+
|      Angina|Disease|count|
+------------+-------+-----+
|Asymptomatic|     No|  160|
|Asymptomatic|    Yes|  137|
+------------+-------+-----+



In [102]:
#moyenne de variable quantitative
data.groupBy('Disease').agg(round(mean('Age'),).alias('Age Average')).show()
data.groupBy('Disease').agg(round(mean('Blood_Pressure'),2).alias('Blood Pressure Average')).show()
data.groupBy('Disease').agg(round(mean('Cholesterol'),2).alias('Cholesterol Average')).show()

+-------+-----------+
|Disease|Age Average|
+-------+-----------+
|     No|       53.0|
|    Yes|       57.0|
+-------+-----------+

+-------+----------------------+
|Disease|Blood Pressure Average|
+-------+----------------------+
|     No|                129.18|
|    Yes|                134.64|
+-------+----------------------+

+-------+-------------------+
|Disease|Cholesterol Average|
+-------+-------------------+
|     No|             243.49|
|    Yes|             251.85|
+-------+-------------------+



In [103]:
#conversion ton numerical form
indexer1 = StringIndexer(inputCol= "Sex", outputCol= "Sex_Index")
indexer2 = StringIndexer(inputCol= "Angina", outputCol= "Angina_Index")
indexer3 = StringIndexer(inputCol= "Glycemia", outputCol= "Glycemia_Index")
indexer4 = StringIndexer(inputCol= "ECG", outputCol= "ECG_Index")
indexer5 = StringIndexer(inputCol= "Angina_After_Sport", outputCol= "Angina_After_Sport_Index")
indexer6 = StringIndexer(inputCol= "ECG_Slope", outputCol= "ECG_Slope_index")
indexer7 = StringIndexer(inputCol= "Fluoroscopy", outputCol= "Fluoroscopy_Index")
indexer8 = StringIndexer(inputCol= "Thalassaemia", outputCol= "Thalassaemia_Index")
indexer9 = StringIndexer(inputCol= "Disease", outputCol= "label")

assembler = VectorAssembler(
    inputCols=["Sex_Index", "Angina_Index", "Glycemia_Index", "ECG_Index", "Angina_After_Sport_Index", "ECG_Slope_index", "Fluoroscopy_Index", "Thalassaemia_Index", "Age", "Blood_Pressure", "Cholesterol", "Heart_Rate", "ECG_Angina"],
    outputCol="features"
)


In [104]:
lr = LogisticRegression()

pipeline = Pipeline(stages = [indexer1, indexer2, indexer3, indexer4, indexer5, indexer6, indexer7, indexer8, indexer9, assembler, lr])

train_data, test_data = data.randomSplit([0.7, 0.3], seed = 123)

model = pipeline.fit(train_data)



23/11/15 21:55:31 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/11/15 21:55:31 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


In [105]:
train_data.toPandas()

,ID,Age,Sex,Angina,Blood_Pressure,Cholesterol,Glycemia,ECG,Heart_Rate,Angina_After_Sport,ECG_Angina,ECG_Slope,Fluoroscopy,Thalassaemia,Disease
0,1,63.0,Male,Asymptomatic,145.0,233.0,More than 120 mg/dl,Hypertrophy,150.0,No,2.3,Falling,No anomaly,Thalassaemia under control,No
1,2,67.0,Male,Asymptomatic,160.0,286.0,Less than 120 mg/dl,Hypertrophy,108.0,Yes,1.5,Stable,High,No,Yes
2,4,37.0,Male,Asymptomatic,130.0,250.0,Less than 120 mg/dl,Normal,187.0,No,3.5,Falling,No anomaly,No,No
3,5,41.0,Male,Asymptomatic,130.0,204.0,Less than 120 mg/dl,Hypertrophy,172.0,No,1.4,Rising,No anomaly,No,No
4,6,56.0,Male,Asymptomatic,120.0,236.0,Less than 120 mg/dl,Normal,178.0,No,0.8,Rising,No anomaly,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,296,41.0,Male,Asymptomatic,120.0,157.0,Less than 120 mg/dl,Normal,182.0,No,0.0,Rising,No anomaly,No,No
206,297,59.0,Male,Asymptomatic,164.0,176.0,More than 120 mg/dl,Hypertrophy,90.0,No,1.0,Stable,Medium,Thalassaemia under control,Yes
207,300,68.0,Male,Asymptomatic,144.0,193.0,More than 120 mg/dl,Normal,141.0,No,3.4,Stable,Medium,Unstable Thalassaemia,Yes
208,301,57.0,Male,Asymptomatic,130.0,131.0,Less than 120 mg/dl,Normal,115.0,Yes,1.2,Stable,Low,Unstable Thalassaemia,Yes


In [106]:
test_data.toPandas()

,ID,Age,Sex,Angina,Blood_Pressure,Cholesterol,Glycemia,ECG,Heart_Rate,Angina_After_Sport,ECG_Angina,ECG_Slope,Fluoroscopy,Thalassaemia,Disease
0,3,67.0,Male,Asymptomatic,120.0,229.0,Less than 120 mg/dl,Hypertrophy,129.0,Yes,2.6,Stable,Medium,Unstable Thalassaemia,Yes
1,7,62.0,Male,Asymptomatic,140.0,268.0,Less than 120 mg/dl,Hypertrophy,160.0,No,3.6,Falling,Medium,No,Yes
2,10,53.0,Male,Asymptomatic,140.0,203.0,More than 120 mg/dl,Hypertrophy,155.0,Yes,3.1,Falling,No anomaly,Unstable Thalassaemia,Yes
3,13,56.0,Male,Asymptomatic,130.0,256.0,More than 120 mg/dl,Hypertrophy,142.0,Yes,0.6,Stable,Low,Thalassaemia under control,Yes
4,14,44.0,Male,Asymptomatic,120.0,263.0,Less than 120 mg/dl,Normal,173.0,No,0.0,Rising,No anomaly,Unstable Thalassaemia,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,290,56.0,Male,Asymptomatic,120.0,240.0,Less than 120 mg/dl,Normal,169.0,No,0.0,Falling,No anomaly,No,No
83,291,67.0,Male,Asymptomatic,152.0,212.0,Less than 120 mg/dl,Hypertrophy,150.0,No,0.8,Stable,No anomaly,Unstable Thalassaemia,Yes
84,294,63.0,Male,Asymptomatic,140.0,187.0,Less than 120 mg/dl,Hypertrophy,144.0,Yes,4.0,Rising,Medium,Unstable Thalassaemia,Yes
85,298,57.0,Male,Asymptomatic,140.0,241.0,Less than 120 mg/dl,Normal,123.0,Yes,0.2,Stable,No anomaly,Unstable Thalassaemia,Yes


In [111]:
predictions = model.transform(test_data)

predictions_pd = predictions.select("label", "prediction")

predictions_pd.toPandas()

,label,prediction
0,1.0,1.0
1,1.0,1.0
2,1.0,0.0
3,1.0,1.0
4,0.0,0.0
...,...,...
82,0.0,0.0
83,1.0,1.0
84,1.0,1.0
85,1.0,1.0


In [116]:
evaluator = MulticlassClassificationEvaluator(labelCol = "label", predictionCol= "prediction", metricName = "accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy : {:.2f}%".format(accuracy * 100))

Accuracy : 82.76%


In [117]:
predictions.select("label", "prediction").crosstab("label", "prediction").show()

+----------------+---+---+
|label_prediction|0.0|1.0|
+----------------+---+---+
|             1.0| 10| 34|
|             0.0| 38|  5|
+----------------+---+---+

